In [4]:
##do_getrealpath
##%overwritefile
##%file:src/do_getrealpath.py
##%noruncode
##u=os.popen('bash -c "uname"')
    def getrealpath(self,filename):
        u=''
        realpath=filename
        try:
            if self.subsys.startswith('MINGW64') or self.subsys.startswith('CYGWIN'):
                r=os.popen('bash -c "cygpath '+filename+'"')
                realpath=r.read()
        except Exception as e:
            self._logln(""+str(e),3)
        return realpath



[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\kernel\WLScript\src/do_getrealpath.py created successfully


In [5]:
##do_runcode
##%overwritefile
##%file:src/do_wlscript_runcode.py
##%noruncode
    def do_runcode(self,return_code,fil_ename,magics,code, silent, store_history=True,
                    user_expressions=None, allow_stdin=True):
        return_code=return_code
        fil_ename=fil_ename
        bcancel_exec=False
        retinfo=self.get_retinfo()
        retstr=''

        ##代码运行前
        # bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,3,1)
        # if bcancel_exec:return bcancel_exec,retinfo,magics, code,fil_ename,retstr
        # self._logln("The process :"+fil_ename)
        fil_ename=self.getrealpath(fil_ename)
        runprgargs=self.get_magicsSvalue(magics,'runprgargs')
        cmds=[]
        if self.sys=="Windows":
            cmds=['cmd','/c']
        cmds+=['wolframscript']
        if (len(runprgargs)>0):
            cmds+=runprgargs
        cmds+=['-f',fil_ename]
        p = self.create_jupyter_subprocess(cmds+ magics['_st']['args'],cwd=None,shell=False,env=self.addkey2dict(magics,'env'),magics=magics)
        self.g_rtsps[str(p.pid)]=p
        return_code=p.returncode
        ##代码启动后
        # bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,3,2)
        # if bcancel_exec:return bcancel_exec,retinfo,magics, code,fil_ename,retstr
         
        if magics!=None and len(self.addkey2dict(magics,'showpid'))>0:
            self._write_to_stdout("The process PID:"+str(p.pid)+"\n")
        return_code=p.wait_end(magics)
        ##代码运行结束
        # self.cleanup_files()
        if return_code != 0:
            self._log("Executable exited with code {}".format(return_code),2)
        return bcancel_exec,retinfo,magics, code,fil_ename,retstr

[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\kernel\WLScript\src/do_wlscript_runcode.py created successfully


In [6]:
##do_compile_code
##%overwritefile
##%file:src/do_wlscript_compilecode.py
##%noruncode
    def do_compile_code(self,return_code,fil_ename,magics,code, silent, store_history=True,
                    user_expressions=None, allow_stdin=True):
        return_code=0
        fil_ename=fil_ename
        sourcefilename=fil_ename
        bcancel_exec=False
        retinfo=self.get_retinfo()
        retstr=''

        return bcancel_exec,retinfo,magics, code,fil_ename,retstr

[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\kernel\WLScript\src/do_wlscript_compilecode.py created successfully


In [7]:
##do_wlscript_create_codefile
##%overwritefile
##%file:src/do_wlscript_create_codefile.py
##%noruncode
    def do_create_codefile(self,magics,code, silent, store_history=True,
                    user_expressions=None, allow_stdin=True):
        return_code=0
        fil_ename=''
        bcancel_exec=False
        retinfo=self.get_retinfo()
        retstr=''
        
        source_file=self.create_codetemp_file(magics,code,suffix='.vbs')
        newsrcfilename=source_file.name
        fil_ename=newsrcfilename
        return_code=True
        
        return bcancel_exec,retinfo,magics, code,fil_ename,retstr

[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\kernel\WLScript\src/do_wlscript_create_codefile.py created successfully


In [8]:
##do_wlscript_preexecute
##%overwritefile
##%file:src/do_wlscript_preexecute.py
##%noruncode
    def do_preexecute(self,code, magics,silent, store_history=True,
                user_expressions=None, allow_stdin=False):
        bcancel_exec=False
        retinfo=self.get_retinfo()


        return_code=0
        fil_ename=''
        return bcancel_exec,retinfo,magics, code

[MyPython] Info:file h:\Jupyter\Myjupyter-kernel\kernel\WLScript\src/do_wlscript_preexecute.py created successfully


In [4]:
## new kernel.py
##%overwritefile
##%file:../../../jupyter-MyWLS-kernel/jupyter_MyWLS_kernel/kernel.py
###%file:wls_kernel.py
##%noruncode
#
#   MyVBScript Jupyter Kernel
#   generated by MyPython
#
##%include:../../src/head.py
##%include:../../src/common.py
##%include:../../src/Mykernel.py
class WLScriptKernel(MyKernel):
    implementation = 'jupyter_MyWLS_kernel'
    implementation_version = '1.0'
    language = 'Wolfram'
    language_version = ''
    language_info = {'name': 'Wolfram Language',
                     'mimetype': 'text/wolfram',
                     'file_extension': '.wls'}
    runfiletype='script'
    banner = "wolframscript kernel.\n" \
             "Uses wolframscript, creates source code files and executables in temporary folder.\n"

    main_head = "\n" \
            "\n" \
            "int main(List<String> arguments){\n"

    main_foot = "\nreturn 0;\n}"
    
##//%include:../../src/comm_attribute.py

    def __init__(self, *args, **kwargs):
        super(WLScriptKernel, self).__init__(*args, **kwargs)
        ## self.kernelinfo='[MyWLSKernel]'
        self._allow_stdin = True
        self.readOnlyFileSystem = False
        self.bufferedOutput = True
        self.linkMaths = True # always link math library
        self.wAll = True # show all warnings by default
        self.wError = False # but keep comipiling for warnings
        self.kernelinfo="[MyWLSKernel{0}]".format(time.strftime("%H%M%S", time.localtime()))
#################
##%include:src/do_getrealpath.py
##%include:src/do_wlscript_runcode.py
##%include:src/do_wlscript_compilecode.py
##%include:src/do_wlscript_create_codefile.py
##%include:src/do_wlscript_preexecute.py



[MyPythonKernel183738] Info:file h:\Jupyter\Myjupyter-kernel\kernel\WLScript\../../../jupyter-MyWLS-kernel/jupyter_MyWLS_kernel/kernel.py created successfully
